In [1]:
from IPython.display import clear_output
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('/home/vco/Projects/pm4py-dcr') # working directory should be pm4py-dcr (the one behind notebooks)
print(os.getcwd())

/home/vco/Projects/pm4py-dcr/notebooks
/home/vco/Projects/pm4py-dcr


In [2]:
import pm4py
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np

from copy import deepcopy
from scipy import stats
from pm4py.algo.discovery.dcr_discover import algorithm as alg
from pm4py.objects.dcr import semantics as dcr_semantics
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
from pm4py.util.benchmarking import benchmark_event_log

In [3]:
event_log_file = '/home/vco/Datasets/Sepsis Cases - Event Log.xes'
dcr_title = 'Sepsis Cases'
result_file_prefix = 'sepsis'
sepsis_log = pm4py.read_xes(event_log_file,return_legacy_log_object=True)
sepsis_reference_dcr, sepsis_la = alg.apply(sepsis_log,alg.DCR_BASIC,timed=False)

/home/vco/.pyenv/versions/3.11.0/envs/pm4py-dcr/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 1050/1050 [00:00<00:00, 1822.96it/s]
/home/vco/Projects/pm4py-dcr/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


In [4]:
event_log = deepcopy(sepsis_log)
dcr_model = deepcopy(sepsis_reference_dcr)
at_least_once = {}
end_excluded = {}
at_least_once_all_traces = set(dcr_model['events'])
end_excluded_all_traces = set(dcr_model['events'])

for trace in event_log:
    executed_events = set()
    im = dcr_model['marking']
    dcr = deepcopy(dcr_model)
    complete=True
    for event in trace:
        executed = dcr_semantics.execute(event['concept:name'], dcr)
        if executed:
            executed_events.add(event['concept:name'])
        complete = complete and event['lifecycle:transition']=='complete'
    if complete:
        fm = dcr['marking']
        excluded_events = im['included'].difference(fm['included'])
        at_least_once_all_traces = at_least_once_all_traces.intersection(executed_events)
        end_excluded_all_traces = end_excluded_all_traces.intersection(excluded_events)

initially_pending = at_least_once_all_traces.union(end_excluded_all_traces)
print(initially_pending)

{'ER Triage', 'ER Registration'}
